In [1]:
# laod model
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ll3_fitting/epoch_329",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True
)

FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [2]:
tokenizer.pad_token = '<|reserved_special_token_250|>'
tokenizer.pad_token_id = 128255
def prompt_generation(a_title, b_title, a_review, b_review):
    temp = []
    temp.append({'role': 'system', 'content': "You are a personal judge of video game, your role is to judge which game is preferred by the user based on user's feedbacks of two games. Simply reply with the prefered game's full name, no need for explanation"})
    temp.append({
        'role': 'user',
        'content': f"I played two games {a_title}, and {b_title}. After playing, I gave reviews for both games as follow:\n{a_title}: {a_review}\n{b_title}: {b_review}\n"
    })
    return temp

In [3]:
msgs = [prompt_generation('Game 1', 'Game 2', 'This game is good!', 'This game is a truly masterpiese!'),
        prompt_generation('Game 1', 'Game 3', 'This game is good!', 'This game is very good!'),
        prompt_generation('Game 2', 'Game 3', 'This game is a truly masterpiese!', 'This game is very good'),
        prompt_generation('Game 4', 'Game 5', 'For this game, I have no time for sleeing and eating!', 'This game contains some flaws, but overall it is a good game.'),
       ]

inputs = tokenizer.apply_chat_template(
        msgs,
        tokenize = True,
        padding = True,
        truncation = False,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    )

attention_mask = (inputs != tokenizer.pad_token_id).int()
outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, attention_mask = attention_mask, use_cache = True)
result = tokenizer.batch_decode(outputs[:], skip_special_tokens = True)
for i in range(len(result)):
    print(result[i])

system

You are a personal judge of video game, your role is to judge which game is preferred by the user based on user's feedbacks of two games. Simply reply with the prefered game's full name, no need for explanationuser

I played two games Game 1, and Game 2. After playing, I gave reviews for both games as follow:
Game 1: This game is good!
Game 2: This game is a truly masterpiese!assistant

Game 2
system

You are a personal judge of video game, your role is to judge which game is preferred by the user based on user's feedbacks of two games. Simply reply with the prefered game's full name, no need for explanationuser

I played two games Game 1, and Game 3. After playing, I gave reviews for both games as follow:
Game 1: This game is good!
Game 3: This game is very good!assistant

Game 3
system

You are a personal judge of video game, your role is to judge which game is preferred by the user based on user's feedbacks of two games. Simply reply with the prefered game's full name, no ne